# Problem 4
For this problem, you need to use the built-in sklearn digits dataset. You can load this data using Sklearn.datasets.load_digits (*, n_class=10,return_X_y=False, as_frame=False) Divide the data into training and test sets using train_test_split and random_state=0 The goal is to train a Random Forest classifier and optimize its performance on this data.
1. Identify the most important parameters that affect the performance of the Random Forest classifier and outline your experimental design (using 4-fold cross validation) to learn the optimal values for these parameters.
2. Analyze the results of the classifier using its optimal parameters and comment on its generalization capability.
3. Visualize and explain the relevant features identified by the Random Forest classifier.
+ Create a white 8x8 image that represents the original 64 features. Map each identified relevant feature to this 2D image and display it using a grey scale that reflects its importance (e.g. 0 most relevant feature and 255  least relevant feature).
4. Identify one misclassified sample from each class (if they exist). Visualize each misclassified sample as an 8x8 image, and use its nearest neighbors and the learned important features to explain why it was misclassified.

Hint: for examples on how to read this data and visualize it, check
https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glrauto-examples-classification-plot-digits-classification-py

## Import and setup data

In [ ]:
import heapq

import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn_evaluation
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

digits = sklearn.datasets.load_digits(n_class=10, return_X_y=False, as_frame=False)
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

X_train, X_test, y_train, y_test = train_test_split(
    data,
    digits.target,
    random_state=0
)
print(len(X_train))
print(len(X_test))

## Train Random Forest classifier

### General methods for training

In [ ]:
np.random.seed(42)


def grid_search_random_forest(
        n_estimators,
        max_features,
        max_depth=None,
        max_leaf_nodes=None
):
    parameters = {
        'n_estimators': n_estimators,
        'max_features': max_features,
    }
    include_additional_parameters(max_depth, max_leaf_nodes, parameters)

    classifier = RandomForestClassifier(random_state=0)

    grid_search = GridSearchCV(
        classifier,
        parameters,
        cv=4,
        return_train_score=True,
    )

    grid_search.fit(X_train, y_train)
    print_grid_search_results(grid_search, parameters)
    return grid_search


def print_grid_search_results(grid_search, parameters):
    for parameter in parameters:
        print(f'Best {parameter}:', grid_search.best_params_[parameter])
    predictions = grid_search.predict(X_test)
    print("Accuracy: ", metrics.accuracy_score(y_test, predictions))


def include_additional_parameters(max_depth, max_leaf_nodes, parameters):
    if max_depth is not None:
        parameters.update({'max_depth': max_depth})
    if max_leaf_nodes is not None:
        parameters.update({'max_leaf_nodes': max_leaf_nodes})


def graph_grid_search_random_forest(grid_search):
    print(grid_search.cv_results_)
    sklearn_evaluation.evaluator.plot.grid_search(
        grid_search.cv_results_,
        change=("n_estimators", "max_features")
    )

    plt.title("Random Forest")
    plt.xlabel("n_estimators")
    plt.ylabel("max_features")
    plt.legend()
    plt.show()


def plot_3d_heatmap(grid_results):
    result_params = grid_results.cv_results_['params']
    df = pd.DataFrame(result_params)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = df.loc[:, 'n_estimators']
    y = df.loc[:, 'max_features']
    z = df.loc[:, 'max_depth']
    c = grid_results.cv_results_['mean_test_score']
    img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
    fig.colorbar(img)
    plt.show()


### Start finding optimal parameters
use 4-fold cross validation and heatmaps to determine best params. keep track of times in case coomputing resources in jeopardy

### Trial 1

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(20, 40, 3, dtype=int)
max_features = np.linspace(1, 12, 6, dtype=int)

grid_search = % time grid_search_random_forest( n_estimators, max_features )
graph_grid_search_random_forest(grid_search)

### Trial 2

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(40, 100, 3, dtype=int)
max_features = np.linspace(7, 16, 6, dtype=int)

grid_search_t2 = %time grid_search_random_forest( n_estimators, max_features )
graph_grid_search_random_forest(grid_search_t2)

### Trial 3

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(100, 140, 5, dtype=int)
max_features = np.linspace(5, 9, 5, dtype=int)

grid_search_t3 = %time grid_search_random_forest( n_estimators, max_features )
graph_grid_search_random_forest(grid_search_t3)

### Trial 4 with Pruning

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(50, 90, 5, dtype=int)
max_features = np.linspace(10, 14, 5, dtype=int)
max_depth = np.linspace(5, 50, 5, dtype=int)

grid_search = % time grid_search_random_forest(n_estimators, max_features, max_depth)

### Trial 5 more pruning

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(90, 130, 5, dtype=int)
max_features = np.linspace(6, 10, 5, dtype=int)
max_depth = np.linspace(5, 50, 5, dtype=int)

grid_search_t5_pruning = grid_search_random_forest(n_estimators, max_features, max_depth)

In [ ]:
plot_3d_heatmap(grid_search_t5_pruning)

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(60, 100, 3, dtype=int)
max_features = np.linspace(9, 15, 6, dtype=int)
max_depth = np.linspace(5, 50, 5, dtype=int)

grid_search = % time grid_search_random_forest(n_estimators, max_features, max_depth)

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(60, 100, 3, dtype=int)
max_features = np.linspace(9, 15, 3, dtype=int)
max_depth = np.linspace(10, 30, 3, dtype=int)
max_leaf_nodes = np.linspace(1, 50, 5, dtype=int)

grid_search = % time grid_search_random_forest(n_estimators, max_features, max_depth, max_leaf_nodes)

In [ ]:
np.random.seed(42)

n_estimators = np.linspace(60, 100, 3, dtype=int)
max_features = np.linspace(12, 18, 3, dtype=int)
max_depth = np.linspace(5, 15, 3, dtype=int)
max_leaf_nodes = np.linspace(40, 140, 5, dtype=int)

grid_search = %time grid_search_random_forest(n_estimators, max_features, max_depth, max_leaf_nodes)

## Test the trained model
use the optimal hyperparameters to set on a final random forest to use throughout the subsequent modeling

In [ ]:
np.random.seed(42)

rf = RandomForestClassifier(
    n_estimators=120,
    max_features=8,
    max_depth=16,
    oob_score=True
)
rf.fit(X_train, y_train)
y_predictions = rf.predict(X_test)
print("Train: ", rf.score(X_train, y_train))
print("Test: ", rf.score(X_test, y_test))

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
print("Train: ", dt.score(X_train, y_train))
print("Test: ", dt.score(X_test, y_test))

## Feature Importance

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
forest_importances = pd.Series(importances)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

plot the feature importance
Plot the forest feature importances via mean decrease in impurity

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(4)
fig.set_figwidth(4)
plt.bar([x for x in range(len(importances))], importances)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.show()

collect most important feature and sum their importance

In [ ]:
import heapq

count_over_threshold = sum(map(lambda x: x >= 0.03, importances))
most_important = heapq.nlargest(count_over_threshold, importances)
least_important = heapq.nsmallest(45, importances)

indices_of_most_important_features = []
for item in most_important:
    correct_actual_item_index = np.argwhere(importances == item)
    indices_of_most_important_features.append(correct_actual_item_index[0][0])

print(indices_of_most_important_features)
print(f'Most important count: {len(most_important)}')
print(f'Most important sum: {len(most_important)}')
print('Least 45 sum:', sum(least_important))

### Confusion Matrix
test the viability of the model, accuracy etc

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_predictions)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

## Analyze Misclassified

general functions

In [ ]:
def map_importance_mask_over_digit_image(
        item_to_display_1d,
        mask_over_item_1d,
        expected_result,
        actual_result
):
    importance_mask = mask_over_item_1d.reshape(8, 8)
    display_item_in_8x8(plt, item_to_display_1d, expected_result, actual_result)

    display_mask_over_image(importance_mask)


def display_mask_over_image(importance_mask):
    plt.imshow(
        importance_mask,
        cmap=plt.cm.gray_r,
        interpolation='none',
        vmin=0,
        vmax=1,
        aspect='equal',
        alpha=0
    )

    def rect(pos):
        r = plt.Rectangle(pos - 0.5, 1, 1, facecolor="none", edgecolor="cyan", linewidth=2)
        plt.gca().add_patch(r)

    x, y = np.meshgrid(np.arange(importance_mask.shape[1]), np.arange(importance_mask.shape[0]))
    m = np.c_[x[importance_mask.astype(bool)], y[importance_mask.astype(bool)]]
    for pos in m:
        rect(pos)
    plt.show()


def display_item_in_8x8(plt, item_to_display_1d, expected_result, actual_result):
    misclassified_image = item_to_display_1d.reshape(8, 8)
    _, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
    axes.set_axis_off()
    axes.imshow(
        misclassified_image,
        cmap=plt.cm.gray_r,
        interpolation="nearest",
    )
    ax.set_title(f"Expected: {expected_result}\nActual: {actual_result}")

def map_misclassification_analysis(
        correct_expected_item_index,
        misclassifed_item_index,
        correct_actual_item_index,
):
    map_importance_mask_over_digit_image(
        X_train[correct_expected_item_index],
        importance_mask_1d,
        y_train[correct_expected_item_index],
        y_train[correct_expected_item_index]
    )

    map_importance_mask_over_digit_image(
        X_test[misclassifed_item_index],
        importance_mask_1d,
        y_test[misclassifed_item_index],
        y_predictions[misclassifed_item_index]
    )

    map_importance_mask_over_digit_image(
        X_train[correct_actual_item_index],
        importance_mask_1d,
        y_train[correct_actual_item_index],
        y_train[correct_actual_item_index]
    )

In [ ]:
misclassified_indices = np.where((y_predictions != y_test))
print(misclassified_indices)


def generate_most_important_mask():
    global importance_image, i, importance_mask_1d
    importance_image = []
    for i in range(0, 64):
        importance_image.append(0)
        if indices_of_most_important_features.__contains__(i):
            importance_image[i] = 1
    return np.array(importance_image)

importance_mask_1d = generate_most_important_mask()

In [ ]:
indices_of_correct_8 = np.where((y_predictions == y_test) & (y_predictions == 8))
print(indices_of_correct_8)

indices_of_correct_1 = np.where((y_predictions == y_test) & (y_predictions == 1))
print(indices_of_correct_1)

In [ ]:
np.random.seed(42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print('Train: ', knn.score(X_train, y_train))
print('Test: ', knn.score(X_test, y_test))

# knn_predictions = knn.predict(X_test, y_test)

distances, neighbor_indices = knn.kneighbors(X_test[misclassified_indices[0]])

print(neighbor_indices)
# print(misclassified_indices[0])
v_misclass = np.vstack(misclassified_indices[0])
print(v_misclass)
# neighbor_indices = np.array(neighbor_indices)
# print(neighbor_indices)
# misclass = np.array(misclassified_indices[0])
# print(misclass)
neighbor_map = np.concatenate((v_misclass, neighbor_indices), 1)
neighbor_map

In [ ]:
indices_of_misclassified_by_number = np.where(
    (y_predictions != y_test) & (y_test == 9)
)
print(indices_of_misclassified_by_number)

show the most important features heatmap and the feature mask

In [ ]:
no_mask = np.array(np.zeros(64))
map_importance_mask_over_digit_image(
    importances,
    no_mask,
    '0-9',
    '0-9'
)

map_importance_mask_over_digit_image(
    importances,
    importance_mask_1d,
    '0-9',
    '0-9'
)

In [ ]:
indices = neighbor_indices

print(y_test[misclassified_indices[0]])
columns = ["Misclassified Test idx", "True Class", "Pred Class"]
for i in range(3):
    columns += ["Neigbor#{}_idx".format(i + 1), "Neigbor#{}_True Class".format(i + 1),
        "Neigbor#{}_Distance".format(i + 1)]

print(columns)
df = pd.DataFrame(columns=columns)

df["Misclassified Test idx"] = misclassified_indices[0]
df["True Class"] = y_test[misclassified_indices[0]]
df["Pred Class"] = y_predictions[misclassified_indices[0]]
for i in range(3):
    df["Neigbor#{}_idx".format(i + 1)] = indices[:, i]
    df["Neigbor#{}_True Class".format(i + 1)] = y_train[indices[:, i]]
    df["Neigbor#{}_Distance".format(i + 1)] = np.around(distances[:, i], decimals=2)

df

In [ ]:
# (expect_true_neighbor, actual_false, actual_neighbor)


Analyze Class 2

In [ ]:
# (expect_true_neighbor, actual_false, actual_neighbor)
indices_of_correct_number = np.where(y_train == 0)
print(indices_of_correct_number)

map_misclassification_analysis(1051, 117, 18)

Analyze Class 4

In [ ]:
indices_of_correct_number = np.where(y_train == 3)
print(indices_of_correct_number)

map_misclassification_analysis(120, 378, 36)

In [ ]:
indices_of_correct_number = np.where(y_train == 4)
print(indices_of_correct_number)

map_misclassification_analysis(41, 315, 418)

In [ ]:
indices_of_correct_number = np.where(y_train == 5)
print(indices_of_correct_number)

map_misclassification_analysis(12, 56, 1145)

In [ ]:
indices_of_correct_number = np.where(y_train == 1)
print(indices_of_correct_number)

map_misclassification_analysis(1192, 124, 25)

In [ ]:
indices_of_correct_number = np.where(y_train == 5)
print(indices_of_correct_number)

map_misclassification_analysis(1275, 130, 12)